## IMPORT LIB

In [19]:
import pandas as pd 
import numpy as np

## TOY DATA

In [20]:
toyDataMatriks = [
    [3,	0,	0,	0,	0,	0,	2],
    [1,	2,	4,	2,	2,	0,	0],
    [0,	4,	1,	0,	0,	2,	4],
    [0,	0,	0,	0,	3,	2,	4]
]

In [21]:
# toydata to dataframe

dfToyData = pd.DataFrame(toyDataMatriks, columns=['item-1', 'item-2', 'item-3', 'item-4','item-5','item-6', 'item-7'], index=['user-1', 'user-2', 'user-3', 'user-4'])
dfToyData

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,3,0,0,0,0,0,2
user-2,1,2,4,2,2,0,0
user-3,0,4,1,0,0,2,4
user-4,0,0,0,0,3,2,4


## MEAN RATING

In [22]:
def meanRating(rating):
    num = rating.sum(axis=1)
    den = np.count_nonzero(rating, axis=1)
    # menghindari pembagian dengan 0
    calCulatedMean  = np.where(den != 0, num / den, 0)
    # menghitung mean
    meanRatingUser = pd.DataFrame(calCulatedMean, index=rating.index, columns=['meanRating'])

    return meanRatingUser

In [23]:
cal_MeanRatingUser = meanRating(dfToyData)
cal_MeanRatingUser

,meanRating
user-1,2.50
user-2,2.20
user-3,2.75
user-4,3.00


In [24]:
cal_MeanRatingItem = meanRating(dfToyData.T)
cal_MeanRatingItem

,meanRating
item-1,2.000000
item-2,3.000000
item-3,2.500000
item-4,2.000000
item-5,2.500000
item-6,2.000000
item-7,3.333333


## MEAN CENTERD

In [25]:
def meanCenterd(rating, meanRating):
    npRatingMatriks = np.array(rating)
    npMeanRatingUser = np.array(meanRating)
    # !0
    meanCenteredUser = np.where(npRatingMatriks!=0, npRatingMatriks - npMeanRatingUser,0)
    dfMeanCenteredUser = pd.DataFrame(meanCenteredUser, index=rating.index, columns=rating.columns)

    return dfMeanCenteredUser

In [26]:
cal_MeanCenteredUser = meanCenterd(dfToyData, cal_MeanRatingUser)
cal_MeanCenteredUser

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,0.5,0.00,0.00,0.0,0.0,0.00,-0.50
user-2,-1.2,-0.20,1.80,-0.2,-0.2,0.00,0.00
user-3,0.0,1.25,-1.75,0.0,0.0,-0.75,1.25
user-4,0.0,0.00,0.00,0.0,0.0,-1.00,1.00


In [27]:
cal_MeanCenteredItem = meanCenterd(dfToyData.T, cal_MeanRatingItem)
cal_MeanCenteredItem


,user-1,user-2,user-3,user-4
item-1,1.000000,-1.0,0.000000,0.000000
item-2,0.000000,-1.0,1.000000,0.000000
item-3,0.000000,1.5,-1.500000,0.000000
item-4,0.000000,0.0,0.000000,0.000000
item-5,0.000000,-0.5,0.000000,0.500000
item-6,0.000000,0.0,0.000000,0.000000
item-7,-1.333333,0.0,0.666667,0.666667


In [28]:
cal_MeanCenteredItemTranpose = cal_MeanCenteredItem.T
cal_MeanCenteredItemTranpose

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,1.0,0.0,0.0,0.0,0.0,0.0,-1.333333
user-2,-1.0,-1.0,1.5,0.0,-0.5,0.0,0.000000
user-3,0.0,1.0,-1.5,0.0,0.0,0.0,0.666667
user-4,0.0,0.0,0.0,0.0,0.5,0.0,0.666667


## USER-BASED

In [29]:
def simRJUser(rating, penggunaU, penggunaV):
    # mendapatkan item yang diberi rating oleh penggunaU dan penggunaV
    # itemRatedPenggunaU = set(np.where(rating.loc[penggunaU, :] != 0)[0])
    itemRatedPenggunaU = set(rating.columns[rating.iloc[penggunaU, : ] != 0])
    # print(itemRatedPenggunaU)
    itemRatedPenggunav = set(rating.columns[rating.iloc[penggunaV, :] != 0])
    # print(itemRatedPenggunav)
    # mencari irisan dari item yang diberi rating oleh penggunaU dan penggunaV
    intersection = len(itemRatedPenggunaU.intersection(itemRatedPenggunav))
    # print(intersection)
    iPenggunaU = len(itemRatedPenggunaU)
    # print(iPenggunaU)
    iPenggunaV = len(itemRatedPenggunav)
    # print(iPenggunaV)
    # menghitung similarity
    if intersection != 0:
        rumusSimRJUser = 1/(1+(1/intersection)+(iPenggunaU/(1+iPenggunaU))+(iPenggunaV/(1+iPenggunaV)))
        # print(1/intersection)
        # print(iPenggunaU/1+iPenggunaU)
        # print(iPenggunaV/1+iPenggunaV)
    else:
        rumusSimRJUser = 0
    
    return rumusSimRJUser

In [30]:
# menghitung similarity antar pengguna
cal_simRJUser = simRJUser(dfToyData,0, 2)
print(f"similarity user-1 dan user-2 = {cal_simRJUser}")

similarity user-1 dan user-2 = 0.28846153846153844


In [31]:
# fungi untuk menghitung semua user

def similarityRJUser(rating):
    jumlahPengguna = rating.shape[0]

    # membuat matriks kosong untuk menyimpan hasil similarity
    simMatriks = np.zeros((jumlahPengguna, jumlahPengguna))
    # menghitung similarity antar pengguna
    for i in range(jumlahPengguna):
        for j in range(jumlahPengguna):
            if i == j:
                simMatriks[i][j] = simRJUser(rating, rating.index[i], rating.index[j])
            elif i != j :
                simMatriks[i][j] = simRJUser(rating, rating.index[i], rating.index[j])
    
    # mengubah matriks similarity menjadi dataframe
    simMatriksDf = pd.DataFrame(simMatriks, index=rating.index, columns=rating.index)

    return simMatriksDf

In [32]:
# fungi untuk menghitung semua user

def similarityAllUser2(rating, jenis = 'user'):
    jumlahPengguna = rating.shape[0]
    # print(f'jumlahPengguna = "{jumlahPengguna}"')
    # membuat matriks kosong untuk menyimpan hasil similarity
    simMatriks = np.zeros((jumlahPengguna, jumlahPengguna))
    # print(simMatriks)
    # menghitung similarity antar pengguna
    for i in range(jumlahPengguna):
        # print(f'loop i = {i}')
        for j in range(i, jumlahPengguna):
            rumusRJMatiks = simRJUser(rating, rating.index[i], rating.index[j])
            simMatriks[i][j] = rumusRJMatiks
            simMatriks[j][i] = rumusRJMatiks

         
            # print(f'simMatriks[{i}][{j}] = {simMatriks[i][j]}')
    
    # mengubah matriks similarity menjadi dataframe
    simMatriksDf = pd.DataFrame(simMatriks, index=rating.index, columns=rating.index)

    return simMatriksDf

## ITEM-BASED

In [33]:
def simRJItem(rating, itemU, itemV):
    # mendapatkan pengguna yang memberi rating pada itemU dan itemV
    userRatedItemU = set(rating.index[rating.iloc[:, itemU] != 0])
    userRatedItemV = set(rating.index[rating.iloc[:, itemV] != 0])
    
    # mencari irisan dari pengguna yang memberi rating pada itemU dan itemV
    intersection = len(userRatedItemU.intersection(userRatedItemV))
    # print("intersection = ", intersection)
    iItemU = len(userRatedItemU)
    iItemV = len(userRatedItemV)

    # menghitung similarity
    if intersection != 0:
        rumusSimRJItem = 1/(1+(1/intersection)+(iItemU/(1+iItemU))+(iItemV/(1+iItemV)))
    elif intersection == 0:
        rumusSimRJItem = 0
    return rumusSimRJItem

In [34]:
calR = simRJItem(dfToyData, 0,1)
print(f"similarity item-3 dan item-3 = {calR}")

similarity item-3 dan item-3 = 0.30000000000000004


In [35]:
# fungi untuk menghitung semua user

def similarityRJItem2(rating):
    jumlahItem = rating.shape[1]

    # membuat matriks kosong untuk menyimpan hasil similarity
    simMatriks = np.zeros((jumlahItem, jumlahItem))
    # menghitung similarity antar pengguna
    for i in range(jumlahItem):
        for j in range(i, jumlahItem):
            rumusRJItem = simRJItem(rating, rating.columns[i], rating.columns[j])
            simMatriks[i][j] = rumusRJItem
            simMatriks[j][i] = rumusRJItem
    # mengubah matriks similarity menjadi dataframe
    simMatriksDf = pd.DataFrame(simMatriks, index=rating.columns, columns=rating.columns)
    return simMatriksDf

In [36]:
calItem = similarityRJItem2(dfToyData)
calItem

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

## KOMBINASI USER ATAU ITEM

In [ ]:
def similarityAllUser2(rating, jenis='user'):
    if jenis == 'user':
        jumlah = rating.shape[0]
        indeks = rating.index
        simMatriks = np.zeros((jumlah, jumlah))
        for i in range(jumlah):
            for j in range(i, jumlah):
                nilaiSim = simRJUser(rating, i, j)
                simMatriks[i][j] = nilaiSim
                simMatriks[j][i] = nilaiSim
        simMatriksDf = pd.DataFrame(simMatriks, index=indeks, columns=indeks)
    elif jenis == 'item':
        jumlah = rating.shape[1]
        kolom = rating.columns
        simMatriks = np.zeros((jumlah, jumlah))
        for i in range(jumlah):
            for j in range(i, jumlah):
                nilaiSim = simRJItem(rating, i, j)
                simMatriks[i][j] = nilaiSim
                simMatriks[j][i] = nilaiSim
        simMatriksDf = pd.DataFrame(simMatriks, index=kolom, columns=kolom)
    return simMatriksDf

In [ ]:
cal_simAllUser = similarityAllUser2(dfToyData, 'user')
cal_simAllUser

,user-1,user-2,user-3,user-4
user-1,0.352941,0.285714,0.288462,0.292683
user-2,0.285714,0.348837,0.319149,0.279070
user-3,0.288462,0.319149,0.350877,0.327869
user-4,0.292683,0.279070,0.327869,0.352941


In [ ]:
cal_simAllItem = similarityAllUser2(dfToyData, 'item')
cal_simAllItem

KeyError: 0

## TOPK

In [ ]:
def topKNeigbors(RatingMatriks, k=2):

    # np array
    ## aturan 1 : mengambil similarity dari matriks rating tertinggi
    ## aturan 2 : abaikan jika dengan dirinya sendiri jadi ambil index lain
    similarityFunction = np.array(similarityRJUser(RatingMatriks))
    # abaikan diagonal dan ambil index lain
    selfNotIncluded = np.where(np.eye(similarityFunction.shape[0])==1, 0, similarityFunction)
    # ambil index dari similarity tertinggi
    topK = np.argsort(selfNotIncluded, axis=1)[:, -k:]
    return topK

In [ ]:
def TopKTetangga(similarity, k):
    return np.argsort(-similarity)[:k]  # Mengambil k tetangga terdekat


## PREDIKSI

In [ ]:
# prediksi user-based

def prediksiCF(RatingMatriks, similarityFunction, mean, meanCen, user="user-2", item="item-2", k=2, jenis="userBased"):

    if jenis == "userBased":
        # mendapatkan item yang diberi rating oleh pengguna
        ratingMatriks = RatingMatriks.loc[:, item].to_numpy()
        # print(ratingMatriks)
        # mendapatan mean centered
        meanCentered = meanCen.loc[:, item].to_numpy()
        # print(meanCentered)
        # mendapatkan similarity
        similarity = similarityFunction.loc[user, :].to_numpy()
        # print(similarity)

    if jenis == "itemBased":
        # mendapatkan item yang diberi rating oleh pengguna
        ratingMatriks = RatingMatriks.loc[user, :].to_numpy()
        # print(ratingMatriks)
        # mendapatan mean centered
        meanCentered = meanCen.loc[user, :].to_numpy()
        # print(meanCentered)
        # mendapatkan similarity
        similarity = similarityFunction.loc[:, item].to_numpy()
        # print(similarity)

    # cek index data user/item yang tidak sama dengan 0
    cek_Index = np.where(ratingMatriks != 0)

    # filter user/item rating, mean centered, similarity, sesuai index data yang tidak bernilai 0
    nilai_meanCentered = np.array(meanCentered)[cek_Index]
    # print("cek nilai meanCenterd", nilai_meanCentered)
    nilai_similarity = similarity[cek_Index]
    # print("cek nilai similarity", nilai_similarity)
    # tetangga terdekat
    indexTetanggaSim = TopKTetangga(nilai_similarity, k)
    # print("cek tetangga ", indexTetanggaSim)

    # pembilang 
    pembilang = np.sum(nilai_meanCentered[indexTetanggaSim] * nilai_similarity[indexTetanggaSim])
    # print("pembilang" ,pembilang)
    # penyebut
    penyebut = np.abs(nilai_similarity[indexTetanggaSim]).sum()
    # print("pennyebuat" ,penyebut)

    # cek apakah penyebut tidak sama dengan 0
    if penyebut != 0:
        if jenis == "userBased":
            # prediksi rating
            rumusPrediksi = mean.loc[user] + (pembilang / penyebut)
        elif jenis == "itemBased":
            # rumusPrediksi rating
            rumusPrediksi = mean.loc[item] + (pembilang / penyebut)
    else:
        if jenis == "userBased":
            rumusPrediksi = mean.loc[user] + 0
        elif jenis == "itemBased":
            rumusPrediksi = mean.loc[item] + 0

    return [item, float(rumusPrediksi)] 
    

In [ ]:
cal_PrediksiRJUser = prediksiCF(dfToyData, cal_simAllUser, cal_MeanRatingUser, cal_MeanCenteredUser, user="user-2", item="item-2", k=2, jenis="userBased")
print(f"prediksi rating user-2 untuk item-2 = {cal_PrediksiRJUser[1]}")

prediksi rating user-2 untuk item-2 = 2.6927608923491544


In [ ]:
cal_PrediksiRJItem = prediksiCF(dfToyData, cal_simAllItem, cal_MeanRatingItem, cal_MeanCenteredItemTranpose, user="user-1", item="item-2", k=2, jenis="itemBased")
print(f"prediksi rating user-1 untuk item-2 = {cal_PrediksiRJItem[1]}")

prediksi rating user-1 untuk item-2 = 2.8477026039030426


## PREDIKSI KESELURUHAN

In [ ]:
def hitungKeseluruhanPrediksi(RatingMatriks, similarityFunction, mean, meanCen, k=2, jenis="userBased"):
    # Membuat DataFrame kosong untuk menyimpan prediksi
    prediksiMatriks = pd.DataFrame(index=RatingMatriks.index, columns=RatingMatriks.columns)
    
    # Iterasi untuk setiap user dan item dalam matriks rating
    for user in RatingMatriks.index:
        for item in RatingMatriks.columns:
            # Cek apakah rating untuk (user, item) adalah 0 atau tidak ada rating
            if RatingMatriks.loc[user, item] == 0:
                # Jika tidak ada rating, prediksi nilai rating menggunakan Collaborative Filtering
                prediksiMatriks.loc[user, item] = prediksiCF(RatingMatriks, similarityFunction, mean, meanCen, user, item, k, jenis)[1]
            else:
                # Jika sudah ada rating, simpan rating asli dari RatingMatriks
                prediksiMatriks.loc[user, item] = RatingMatriks.loc[user, item]

    return prediksiMatriks


In [ ]:
cal_HitungAllPrediksiUserRJ = hitungKeseluruhanPrediksi(dfToyData, cal_simAllUser, cal_MeanRatingUser, cal_MeanCenteredUser, k=2, jenis="userBased")
cal_HitungAllPrediksiUserRJ

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,3,3.028535,2.516344,2.3,2.40121,1.624097,2
user-2,1,2,4,2,2,1.333358,2.623322
user-3,2.357192,4,1,2.55,2.65136,2,4
user-4,2.670217,3.583287,2.882298,2.8,3,2,4


In [ ]:
cal_HitungAllPrediksiItemRJ = hitungKeseluruhanPrediksi(dfToyData, cal_simAllItem, cal_MeanRatingItem, cal_MeanCenteredItemTranpose, k=2, jenis="itemBased")
cal_HitungAllPrediksiItemRJ

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,3,2.847703,2.347703,3.0,2.347703,0.666667,2
user-2,1,2,4,2,2,2.25,2.333333
user-3,1.75,4,1,1.75,2.25,2,4
user-4,2.582307,3.25,2.75,2.5,3,2,4


## HYBRID

In [ ]:
def HybridFiltering(PrediksiUser, PrediksiItem, gamma=0.7):
    # Mengonversi ke numpy array
    prediksiUser = np.array(PrediksiUser)
    prediksiItem = np.array(PrediksiItem)
    # Menghitung prediksi hybrid
    rumusHybrid = gamma * prediksiUser + (1 - gamma) * prediksiItem    
    return rumusHybrid


In [ ]:
cal_HybridFiltering = HybridFiltering(cal_HitungAllPrediksiUserRJ, cal_HitungAllPrediksiItemRJ, gamma=0.7)
cal_HybridFiltering

array([[3.0, 2.9742855286456598, 2.4657519166637725, 2.51,
        2.3851579457628906, 1.3368676875430143, 2.0],
       [1.0, 2.0, 4.0, 2.0, 2.0, 1.6083508859913076, 2.5363252314814817],
       [2.1750345622119815, 4.0, 1.0, 2.3099999999999996,
        2.530952086553323, 2.0, 4.0],
       [2.6438438883883566, 3.483300658978583, 2.8426087314662274, 2.71,
        3.0, 2.0, 4.0]], dtype=object)

## TOPN

In [ ]:
def TopNRecommendation(rekomendasiHybrid, N=5):
    topN_per_user = np.argsort(-rekomendasiHybrid, axis=1)[:N]+1
    # tolist
    topN_per_user = topN_per_user.tolist()

    return topN_per_user


In [ ]:
cal_TopNRecommendation = TopNRecommendation(cal_HybridFiltering, N=3)
cal_TopNRecommendation

[[1, 2, 4, 3, 5, 7, 6], [3, 7, 2, 4, 5, 6, 1], [2, 7, 5, 4, 1, 6, 3]]

## OPSI LAIN

In [ ]:
def simRJDuaUser2(RatingMatriks, userU, userV):
    # Ambil indeks item yang diberi rating oleh masing-masing user (rating ≠ 0)
    
    userRatedU = set(RatingMatriks.columns[RatingMatriks[userU] != 0])
    userRatedV = set(RatingMatriks.columns[RatingMatriks[userV] != 0])

    # Hitung intersection dan panjang masing-masing
    intersection = len(userRatedU & userRatedV)
    lenU, lenV = len(userRatedU), len(userRatedV)

    # Hitung similarity RJ item
    if intersection > 0:
        sim = 1 / (1 + (1 / intersection) + (lenU / 1 + lenU) + (lenV / 1 + lenV))
    else:
        sim = 0
    return sim

In [ ]:
simRjDuasUser = simRJDuaUser2(dfToyData, 'user-1', 'user-2')
simRjDuasUser

KeyError: 'user-1'

In [ ]:
def simRJDuaItem(RatingMatriks, itemI, itemJ):
    # Ambil user yang memberi rating untuk masing-masing item
    ratedI = set(RatingMatriks.index[RatingMatriks[itemI] != 0])
    ratedJ = set(RatingMatriks.index[RatingMatriks[itemJ] != 0])

    # Hitung intersection dan panjang masing-masing
    intersection = len(ratedI & ratedJ)
    lenI, lenJ = len(ratedI), len(ratedJ)

    # Hitung similarity RJ item
    if intersection > 0:
        sim = 1 / (1 + (1 / intersection) + (lenI / 1 + lenI) + (lenJ / 1 + lenJ))
    else:
        sim = 0

    return sim


In [ ]:
# fungi untuk menghitung semua user

def similarityRJItem(rating):
    jumlahItem = rating.shape[1]

    # membuat matriks kosong untuk menyimpan hasil similarity
    simMatriks = np.zeros((jumlahItem, jumlahItem))
    # menghitung similarity antar pengguna
    for i in range(jumlahItem):
        for j in range(i, jumlahItem):
            simMatriks[i][j] = simRJItem(rating, rating.columns[i], rating.columns[j])
            simMatriks[j][i] = simRJItem(rating, rating.columns[i], rating.columns[j])

    
    # mengubah matriks similarity menjadi dataframe
    simMatriksDf = pd.DataFrame(simMatriks, index=rating.columns, columns=rating.columns)

    return simMatriksDf